# Load CSV file and enter pair you want to observe

Tuesday Pair Combinations (name, data column)

P6GR, 113

P2CR, 100

P1ER, 139

P5FR, 206

P3HR, 224

Friday Pair Combinations (name, data column)

P7CR, 100

P11ER, 83

P8IR, 48

In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter1d
from tqdm import tqdm

# Load the data
# change this line depending on tuesday or friday data
df = pd.read_csv('tues_axw_video_data.csv')
#df = pd.read_csv('fri_axw_video_data.csv')

# Extract the time and synchrony data for your pair 
time_data = df['Time']

#enter the column number for the pair you want to look at here
pair = df['224']

# Video with Overlayed Heat Map for one pair

In [2]:
# Apply smoothing
smoothed_sync = gaussian_filter1d(pair, sigma=5)

# Create custom colormap
colors = ['blue', 'cyan', 'yellow', 'red']
n_bins = 100
cmap = LinearSegmentedColormap.from_list('cool_to_warm', colors, N=n_bins)

# Open the video
# change this line depending on day
video = cv2.VideoCapture('TuesdayVideo.mov')
#video = cv2.VideoCapture('FridayVideo.mov')

# Get video properties
fps = video.get(cv2.CAP_PROP_FPS)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Set up the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# name your output file
#change depending on pair
out = cv2.VideoWriter('P3HR.mp4', fourcc, fps, (width, height))

# Heat map dimensions and position
hm_width, hm_height = 100, 100
hm_x, hm_y = width - hm_width - 10, height - hm_height - 10

# Set up progress bar
pbar = tqdm(total=total_frames, desc="Processing video")

frame_count = 0

while True:
    ret, frame = video.read()
    if not ret:
        break

    current_time = frame_count / fps
    if current_time >= time_data.iloc[-1]:
        break

    # Find the closest time index
    time_index = np.searchsorted(time_data, current_time)
    
    # Get the current synchrony value
    current_sync = smoothed_sync[time_index]

    # Create heat map
    heatmap = np.full((hm_height, hm_width, 3), current_sync)
    heatmap = (cmap(heatmap[:,:,0])[:,:,:3] * 255).astype(np.uint8)

    # Add heat map to frame
    frame[hm_y:hm_y+hm_height, hm_x:hm_x+hm_width] = heatmap

    # Add time text below heat map
    cv2.putText(frame, f"Time: {current_time:.2f}s", (hm_x, hm_y+hm_height+20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    # Add synchrony value above heat map
    cv2.putText(frame, f"Sync: {current_sync:.2f}", (hm_x, hm_y-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    out.write(frame)
    frame_count += 1
    
    # Update progress bar
    pbar.update(1)

video.release()
out.release()
cv2.destroyAllWindows()

# Close progress bar
pbar.close()

print("Video processing complete.")

[ WARN:0@581.277] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 545438.342211 ms
[ WARN:0@581.278] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 545440.369843 ms
[ WARN:0@581.278] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 545440.406592 ms
[ WARN:0@581.278] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 545440.419217 ms
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x10e9f4590] moov atom not found
Processing video: 406945it [1:24:38, 80.12it/s]                                 

Video processing complete.


# All pairs together with heat map overlay

In [ ]:
# Load the data Tuesday
#df = pd.read_csv('axw_video_data.csv')

# Load the data Friday
df = pd.read_csv('fri_axw_video_data.csv')

# Extract the time data
time_data = df['Time']

# List of pairs tuesday
#pairs = ['113', '100', '139', '206', '224']

#List of pairs friday
pairs = ['100', '83', '48']

# Custom titles tuesday
#titles = ['P6GR', 'P2CR', 'P1ER', 'P5FR', 'P3HR']

# Custom titles friday
titles = ['P7CR', 'P11ER', 'P8IR']


# Apply smoothing to each pair
smoothed_syncs = {pair: gaussian_filter1d(df[pair], sigma=5) for pair in pairs}

# Create the custom colormap
colors = ['blue', 'cyan', 'yellow', 'red']
n_bins = 100
cmap = LinearSegmentedColormap.from_list('cool_to_warm', colors, N=n_bins)

# Open the video
#video = cv2.VideoCapture('TuesdayVideo.mov')
video = cv2.VideoCapture('FridayVideo.mov')


# Get video properties
fps = video.get(cv2.CAP_PROP_FPS)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Set up the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#out = cv2.VideoWriter('allTuesday.mp4', fourcc, fps, (width, height))
out = cv2.VideoWriter('allFriday.mp4', fourcc, fps, (width, height))

# Heat map dimensions and position
hm_width, hm_height = 100, 100
hm_y = 50  # Adjusted position for better visibility

# Set up progress bar
pbar = tqdm(total=total_frames, desc="Processing video")

frame_count = 0

while True:
    ret, frame = video.read()
    if not ret:
        break

    current_time = frame_count / fps
    if current_time >= time_data.iloc[-1]:
        break

    # Find the closest time index
    time_index = np.searchsorted(time_data, current_time)
    
    # Process each pair
    for i, (pair, title) in enumerate(zip(pairs, titles)):
        current_sync = smoothed_syncs[pair][time_index]

        # Create heat map
        heatmap = np.full((hm_height, hm_width, 3), current_sync)
        heatmap = (cmap(heatmap[:,:,0])[:,:,:3] * 255).astype(np.uint8)

        # Calculate x position for each heatmap
        hm_x = 10 + i * (hm_width + 10)

        # Add heat map to frame
        frame[hm_y:hm_y+hm_height, hm_x:hm_x+hm_width] = heatmap

        # Add custom title above each heat map
        cv2.putText(frame, title, (hm_x, hm_y-15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
        
        # Add synchrony value below each heat map
        cv2.putText(frame, f"{current_sync:.2f}", (hm_x, hm_y+hm_height+20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    out.write(frame)
    frame_count += 1
    
    # Update progress bar
    pbar.update(1)

video.release()
out.release()

# Close progress bar
pbar.close()

print("Video processing complete with custom titles and synchrony values.")

# All other static visuals

In [4]:
# Pair P6GR Segments
# change these depending on what pair you're looking at

segments = [
    (0, 435),     # 00:00:00 - 00:07:15
    (435, 525),  # 00:07:15 - 00:08:45
    (525, 593), # 00:08:45 - 00:09:53 
    (593, 695), # 00:09:53 - 00:11:35 IN
    (695, 900), # 00:11:35 - 00:15:00
    (900, 1207), # 00:15:00 - 00:20:07
    (1207, 1405), # 00:20:07- 00:23:25
    (1405, 1525), # 00:23:25 - 00:25:25 IN
    (1525, 1640), # 00:25:25- 00:27:20
    (1640, 1917), # 00:27:20 - 00:31:57
    (1917, 1965), # 00:31:57 -  00:32:45
    (1965, 2070), # 00:32:45 - 00:34:30 IN
    (2070, 2505), # 00:34:30 -  00:41:45
    (2505, 2893), # 00:41:45 - 00:48:13
    (2893, 3202), # 00:48:13 - 00:53:22
    (3202, 3285), # 00:53:22 - 00:54:45 IN
    (3285, 3378), # 00:54:45 - 00:56:18
    (3378, 3722), # 00:56:18 - 01:02:02
    (3722, 3860), # 01:02:02 - 01:04:20
    (3860, 3955), # 01:04:20 - 01:05:55 IN
    (3955, 4363), # 01:05:55 - 01:12:43
    (4363, 4646), # 01:12:43 - 01:17:26
    (4646, 5396), # 01:17:26 - 01:29:56
    (5396, 5493) # 01:29:56 - 01:31:33
]

segment_labels = [
    'Intro (Hello)',
    'Group Activity #1',
    'Game #1 (out)',
    'Game #1 (in)',
    'Game #1 (out)',
    'Group Activity #2',
    'Game #2 (out)',
    'Game #2 (in)',
    'Game #2 (out)',
    'Group Activity #3',
    'Game #3 (out)',
    'Game #3 (in)',
    'Game #3 (out)',
    'Individual Activity #1',
    'Game #4 (out)',
    'Game #4 (in)',
    'Game #4 (out)',
    'Group Activity #4',
    'Game #5 (out)',
    'Game #5 (in)',
    'Game #5 (out)',
    'Group Activity #5',
    'Individual Activity #2',
    'Outro (Goodbye)'
]

# Synchrony Plot and Key Moments

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file
df = pd.read_csv('tues_axw_video_data.csv')

def analyze_and_create_table_with_segments(df, column, segments, exclude_last_minutes=15, interval_seconds=30):
    # Convert Time to seconds if it's not already
    if df['Time'].max() < 1000:  # Assuming if max is less than 1000, it's in minutes
        df['Time'] = df['Time'] * 60

    # Calculate the total duration and the cutoff time
    total_duration = df['Time'].max()
    cutoff_time = total_duration - (exclude_last_minutes * 60)

    # Filter out the last 15 minutes
    df_filtered = df[df['Time'] <= cutoff_time].copy()

    # Find peaks in the raw data for each interval
    intervals = []
    for start_time in range(0, int(cutoff_time), interval_seconds):
        end_time = start_time + interval_seconds
        interval_data = df_filtered[(df_filtered['Time'] >= start_time) & (df_filtered['Time'] < end_time)]
        if not interval_data.empty:
            peak_index = interval_data[column].idxmax()
            peak_value = interval_data.loc[peak_index, column]
            peak_time = interval_data.loc[peak_index, 'Time']
            intervals.append((start_time, end_time, peak_value, peak_time))

    # Sort intervals by peak value and get top 5
    top_intervals = sorted(intervals, key=lambda x: x[2], reverse=True)[:5]

    # Convert seconds to HH:MM:SS format
    def seconds_to_hms(seconds):
        return f"{int(seconds//3600):02d}:{int((seconds%3600)//60):02d}:{int(seconds%60):02d}"

    # Function to determine which segment a time belongs to
    def get_segment(time):
        for i, (start, end) in enumerate(segments):
            if start <= time < end:
                return segment_labels[i]
        return "Unknown"

    # Create a DataFrame for the table
    df_table = pd.DataFrame(top_intervals, columns=['Start', 'End', 'Peak Value', 'Peak Time'])
    df_table['Start'] = df_table['Start'].apply(seconds_to_hms)
    df_table['End'] = df_table['End'].apply(seconds_to_hms)
    df_table['Peak Time'] = df_table['Peak Time'].apply(seconds_to_hms)
    df_table['Peak Value'] = df_table['Peak Value'].apply(lambda x: f"{x:.4f}")
    df_table['Segment'] = df_table['Peak Time'].apply(lambda x: get_segment(sum(int(i) * 60 ** j for j, i in enumerate(reversed(x.split(':'))))))

    # Create a figure and axis for the table
    fig, ax = plt.subplots(figsize=(12, 4))

    # Hide axes
    ax.axis('off')

    # Create the table
    table = ax.table(cellText=df_table.values,
                     colLabels=df_table.columns,
                     cellLoc='center',
                     loc='center')

    # Set font size
    table.auto_set_font_size(False)
    table.set_fontsize(10)

    # Scale the table to fit the figure
    table.scale(1, 1.5)

    # Add a title
    plt.title("P6GR Top 5 Intervals of Synchrony", fontsize=16, pad=20)

    # Save the table as an image
    plt.savefig('P6GR_top_5.png', bbox_inches='tight', dpi=300)
    plt.close()


    # Plot the results
    plt.figure(figsize=(14, 8))
    plt.plot(df_filtered['Time'] / 60, df_filtered[column], label='Raw data', alpha=0.8)
    for _, _, peak_value, peak_time in top_intervals:
        plt.scatter([peak_time / 60], [peak_value], color='red', s=100, zorder=5)

    # Add segment boundaries and labels
    for i, (start, end) in enumerate(segments):
        if i > 0:  # Skip the first start as it's 0
            plt.axvline(x=start/60, color='green', linestyle='--', alpha=0.5)
        mid_point = (start + end) / 2 / 60
        plt.text(mid_point, plt.ylim()[0] - 0.05, segment_labels[i], ha='right', va='top', rotation=45, fontsize=10)

    plt.title(f'Synchrony Analysis for Column {column} with Segments')
    plt.xlabel('Time (minutes)')
    plt.ylabel('Synchrony')
    plt.legend()
    plt.tight_layout()
    
    # Adjust the bottom margin to make room for the labels
    plt.subplots_adjust(bottom=0.2)
    
    plt.savefig('P6GRsynchrony_line.png')
    plt.close()





# Heat map by activity type

In [ ]:
def calculate_and_plot_average_synchrony_heatmap_with_labels(df, column, intervals, labels):
    # Convert Time to seconds if it's not already
    if df['Time'].max() < 1000:  # Assuming if max is less than 1000, it's in minutes
        df['Time'] = df['Time'] * 60

    # Calculate average synchrony for each specified interval
    averages = []
    for start_time, end_time in intervals:
        interval_data = df[(df['Time'] >= start_time) & (df['Time'] < end_time)]
        if not interval_data.empty:
            avg_synchrony = interval_data[column].mean()
            averages.append((start_time, end_time, avg_synchrony))
        else:
            averages.append((start_time, end_time, None))

    # Create a DataFrame for the results
    df_averages = pd.DataFrame(averages, columns=['Start', 'End', 'Average Synchrony'])
    df_averages['Average Synchrony'] = df_averages['Average Synchrony'].apply(lambda x: x if x is not None else 0)
    df_averages['Duration'] = df_averages['End'] - df_averages['Start']

    # Add segment names to the DataFrame
    df_averages['Segment Name'] = labels

    # Create a new DataFrame for the CSV file
    df_csv = pd.DataFrame({
        'Segment Name': df_averages['Segment Name'],
        'Segment Times': df_averages.apply(lambda row: f"{row['Start']}-{row['End']}", axis=1),
        'Average Synchrony': df_averages['Average Synchrony']
    })

    # Save the new DataFrame to a CSV file
    csv_filename = 'P6GR_segment_synchrony_data.csv'
    df_csv.to_csv(csv_filename, index=False)
    print(f"New CSV file '{csv_filename}' has been created with segment names, times, and average synchrony.")


    print("Average synchrony for specified intervals:")
    print(df_averages.to_string(index=False))

    # Create a custom colormap
    colors = ['red','yellow', 'cyan', 'blue']
    n_bins = 100
    cmap = LinearSegmentedColormap.from_list('warm_to_cool', colors, N=n_bins)

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(20, 6))

    # Calculate total duration
    total_duration = df_averages['Duration'].sum()

    # Plot rectangles for each interval
    for i, row in df_averages.iterrows():
        start = row['Start']
        duration = row['Duration']
        synchrony = row['Average Synchrony']
        # Normalize synchrony value to the range of the data
        norm_synchrony = (synchrony - df_averages['Average Synchrony'].min()) / (df_averages['Average Synchrony'].max() - df_averages['Average Synchrony'].min())
        rect = Rectangle((start, 0), duration, 1, facecolor=cmap(norm_synchrony))
        ax.add_patch(rect)
        
        # Add vertical text label
        mid_point = start + duration / 2
        ax.text(mid_point, 0.5, labels[i], ha='center', va='center', rotation=90, fontsize=8, color='black')

    # Set the limits of the plot
    ax.set_xlim(0, total_duration)
    ax.set_ylim(0, 1)

    # Remove y-axis ticks
    ax.set_yticks([])

    # Set x-axis ticks to show time in minutes
    x_ticks = np.arange(0, total_duration + 1, 600)  # Every 10 minutes
    ax.set_xticks(x_ticks)
    ax.set_xticklabels([f'{int(x/60)}' for x in x_ticks])

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=df_averages['Average Synchrony'].min(), vmax=df_averages['Average Synchrony'].max()))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, label='Average Synchrony', orientation='vertical', aspect=30)
    cbar.ax.yaxis.set_label_position('left')
    cbar.ax.yaxis.labelpad = 15

    plt.title('Average Synchrony By Activity')
    plt.xlabel('Time (minutes)')
    plt.tight_layout()
    plt.savefig('P6GR_average_synchrony_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
# run visuals for pair P6GR
selected_column = "113"
analyze_and_create_table_with_segments(df, selected_column, segments, exclude_last_minutes=15, interval_seconds=30)
calculate_and_plot_average_synchrony_heatmap_with_labels(df, selected_column, segments, segment_labels)

# Tuesday and Friday Segment code for each pair so that any pair can be run:

## Tuesday Segment Labels:

## Tuesday time for each pair

# Friday segment labels

## Friday time for each pair